In [16]:
# 打印模型形状
import onnx

# Load the ONNX model
model = onnx.load("example.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph test_matmul_add_mode (
  %x[FLOAT, 1x10]
) optional inputs with matching initializers (
  %matmul_weight[FLOAT, 10x1]
  %add_bias[FLOAT, 1000000]
) {
  %matmul_out = MatMul(%x, %matmul_weight)
  %add_out = Add(%matmul_out, %add_bias)
  return %add_out
}


In [45]:
# 生成极端matmul-add模型

import onnx
import numpy as np
# 定义matmul节点的属性
matmul_attr = {}
# 定义add节点的属性
add_attr = {}
# 定义输入张量
x = np.random.randn(1, 10).astype(np.float32)
# 构建matmul节点的输入张量
matmul_weight = np.random.randn(10, 1).astype(np.float32)
# 构建add节点的输入张量
add_bias = np.random.randn(1000000).astype(np.float32)
# 构建matmul节点
matmul_node = onnx.helper.make_node('MatMul', inputs=['x', 'matmul_weight'], outputs=['matmul_out'], **matmul_attr)
# 构建add节点
add_node = onnx.helper.make_node('Add', inputs=['matmul_out', 'add_bias'], outputs=['add_out'], **add_attr)
# 构建onnx模型
onnx_graph = onnx.helper.make_graph(
    nodes=[matmul_node, add_node],
    name='test_matmul_add_mode',
    inputs=[onnx.helper.make_tensor_value_info('x', onnx.TensorProto.FLOAT, [1, 10]),
            onnx.helper.make_tensor_value_info('matmul_weight', onnx.TensorProto.FLOAT, [10, 1]),
            onnx.helper.make_tensor_value_info('add_bias', onnx.TensorProto.FLOAT, [1000000])],
    outputs=[onnx.helper.make_tensor_value_info('add_out', onnx.TensorProto.FLOAT, [1, 1])],
    initializer=[onnx.helper.make_tensor('matmul_weight', onnx.TensorProto.FLOAT, [10, 1], matmul_weight),
                  onnx.helper.make_tensor('add_bias', onnx.TensorProto.FLOAT, [1000000], add_bias)]
)
onnx_model = onnx.helper.make_model(onnx_graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
                                    
onnx.checker.check_model(onnx_model)
# 保存模型
onnx.save(onnx_model, 'example.onnx')

In [29]:
# 生成正常matmul-add模型

import onnx
import numpy as np
# 定义matmul节点的属性
matmul_attr = {}
# 定义add节点的属性
add_attr = {}
# 定义输入张量
x = np.random.randn(10, 10).astype(np.float32)
# 构建matmul节点的输入张量
matmul_weight = np.random.randn(10, 10).astype(np.float32)
# 构建add节点的输入张量
add_bias = np.random.randn(10).astype(np.float32)
# 构建matmul节点
matmul_node = onnx.helper.make_node('MatMul', inputs=['x', 'matmul_weight'], outputs=['matmul_out'], **matmul_attr)
# 构建add节点
add_node = onnx.helper.make_node('Add', inputs=['matmul_out', 'add_bias'], outputs=['add_out'], **add_attr)
# 构建onnx模型
onnx_graph = onnx.helper.make_graph(
    nodes=[matmul_node, add_node],
    name='test_matmul_add_mode',
    inputs=[onnx.helper.make_tensor_value_info('x', onnx.TensorProto.FLOAT, [10, 10]),
            onnx.helper.make_tensor_value_info('matmul_weight', onnx.TensorProto.FLOAT, [10, 10]),
            onnx.helper.make_tensor_value_info('add_bias', onnx.TensorProto.FLOAT, [10])],
    outputs=[onnx.helper.make_tensor_value_info('add_out', onnx.TensorProto.FLOAT, [10, 10])],
    initializer=[onnx.helper.make_tensor('matmul_weight', onnx.TensorProto.FLOAT, [10, 10], matmul_weight),
                  onnx.helper.make_tensor('add_bias', onnx.TensorProto.FLOAT, [10], add_bias)]
)
onnx_model = onnx.helper.make_model(onnx_graph, opset_imports=[onnx.helper.make_opsetid("", 14)])
                                    
onnx.checker.check_model(onnx_model)
# 保存模型
onnx.save(onnx_model, 'example.onnx')

In [30]:
import onnxruntime as ort
import numpy as  np

In [46]:
# 优化onnx模型，需要融合

# 加载onnx模型
model_path = 'example.onnx'
sess_options = ort.SessionOptions()
sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED  
sess_options.optimized_model_filepath = 'optimized.onnx'
sess = ort.InferenceSession(model_path, sess_options=sess_options)
# 运行推理
# x = np.random.randn(1, 10).astype(np.float32)
# result = sess.run(None, {'x': x})
# 打印输出
# print(result)

2023-04-17 08:01:50.193523728 [W:onnxruntime:, graph.cc:1231 Graph] Initializer matmul_weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2023-04-17 08:01:50.193564246 [W:onnxruntime:, graph.cc:1231 Graph] Initializer add_bias appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2023-04-17 08:01:50.193736635 [W:onnxruntime:, graph.cc:108 MergeShapeInfo] Error merging shape info for output. 'add_o

In [47]:
# 打印出融合的模型
import onnx

# Load the ONNX model
model = onnx.load("optimized.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph test_matmul_add_mode (
  %x[FLOAT, 1x10]
) optional inputs with matching initializers (
  %matmul_weight[FLOAT, 10x1]
  %add_bias[FLOAT, 1000000]
) {
  %matmul_out = MatMul(%x, %matmul_weight)
  %add_out = Add(%matmul_out, %add_bias)
  return %add_out
}
